<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/04_hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-07-22 09:53:33--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11848 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.57K  --.-KB/s    in 0s      

2022-07-22 09:53:33 (128 MB/s) - ‘utils.py’ saved [11848/11848]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'data')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 20092 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model(img_size=IMG_SIZE)

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [10]:
callbacks = hsv_creator.get_callbacks()

In [11]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
628/628 [==============================] - 2429s 4s/step - loss: 15.2342 - auc: 0.7695 - accuracy: 0.6905 - val_loss: 0.8137 - val_auc: 0.8226 - val_accuracy: 0.7413
Epoch 2/15
628/628 [==============================] - 289s 458ms/step - loss: 8.5061 - auc: 0.9038 - accuracy: 0.8236 - val_loss: 0.7877 - val_auc: 0.8509 - val_accuracy: 0.7634
Epoch 3/15
628/628 [==============================] - 290s 460ms/step - loss: 6.2741 - auc: 0.9377 - accuracy: 0.8639 - val_loss: 0.8366 - val_auc: 0.8561 - val_accuracy: 0.7874
Epoch 4/15
628/628 [==============================] - 287s 456ms/step - loss: 4.6552 - auc: 0.9620 - accuracy: 0.8986 - val_loss: 1.0316 - val_auc: 0.8156 - val_accuracy: 0.7554
Epoch 5/15
628/628 [==============================] - 289s 458ms/step - loss: 3.6101 - auc: 0.9712 - accuracy: 0.9169 - val_loss: 0.9367 - val_auc: 0.8300 - val_accuracy: 0.7671
Epoch 6/15
628/628 [==============================] - 289s 458ms/step - loss: 2.5430 - auc: 0.9846 - accuracy: 

In [12]:
!cp ./logs/checkpoints/HSV-model.h5 ./drive/MyDrive/checkpoints/hsv-model-aug.h5